In [1]:
import os
import pandas as pd
import numpy as np
import re
import string

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.corpus import wordnet
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
from nltk import word_tokenize, pos_tag, ne_chunk

import gensim
from gensim import corpora, models, similarities

from collections import defaultdict

In [2]:
dataset = pd.read_csv('Train.csv')

### Cleaning the given data

In [3]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [4]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [5]:
dataset[['question']]= pd.DataFrame(dataset.question.apply(round1))
dataset[['answer_text']]= pd.DataFrame(dataset.answer_text.apply(round1))
dataset[['distractor']]= pd.DataFrame(dataset.distractor.apply(round1))
dataset[['question']]= pd.DataFrame(dataset.question.apply(round2))
dataset[['answer_text']]= pd.DataFrame(dataset.answer_text.apply(round2))
dataset[['distractor']]= pd.DataFrame(dataset.distractor.apply(round2))
dataset.head()

,question,answer_text,distractor
0,meals can be served,in rooms at p m,outside the room at p m in the dining room a...
1,it can be inferred from the passage that,the local government can deal with the problem...,if some tragedies occur again relevant depar...
2,the author called tommy s parents in order to,help them realize their influence on tommy,blame tommy for his failing grades blame tommy...
3,it can be inferred from the passage that,the writer is not very willing to use idioms,idioms are the most important part in a langua...
4,how can we deal with snake wounds according to...,stay calm and do nt move,cut the wound and suck the poison out


In [6]:
sample = dataset

In [7]:
x = sample['question'].values.tolist()
y = sample['answer_text'].values.tolist()
z = sample['distractor'].values.tolist()

In [8]:
total = x+y+z

In [9]:
# total_cor = [nltk.word_tokenize(sent) for sent in total]

In [10]:
total_all = []
for z in total:
    total_all.append(z.split())

### MODEL Generated from train data

In [37]:
model = gensim.models.Word2Vec(total_all, min_count=1)

In [41]:
#FOR SIMILARITY CHECK(%)
# def similarity(word1,word2):
#     if (wordnet.synsets(word1)):
#         syn1 = (wordnet.synsets(word1))[0]
#     else:
#         return 0
#     if (wordnet.synsets(word2)):
#         syn2 = (wordnet.synsets(word2))[0]
#     else:
#         return 0
    
#     return syn1.wup_similarity(syn2)
def similarity(word1,word2):
    return model.wv.similarity(word1,word2)

In [13]:
test_data = pd.read_csv('Results.csv')

In [14]:
#wanted pattern        
pattern = re.compile(r'^N\w|V\w|N\w\w|V\w\w|J\w|J\w\w|R\w|R\w\w')
def token_tag(word_tag):
    sample= {}
    idx = []
    result = []
    pos = dict(word_tag)
    for s,t in enumerate(pos.values()):
        if pattern.match(t):
            idx.append(s)
    for s in idx:
        result.append(word_tag[s][0])
    return result,idx

In [52]:
def import_data(df):
    answer_text = df[['answer_text']].values.tolist()
    l = 2
    distractors = []
    for a in answer_text:
        individual = word_tokenize(str(' '.join(a)))
        dict_s = {}
        for b,c in enumerate(individual):
            dict_s[b] = c
        token = word_tokenize(str(' '.join(a)))
        token_pos = nltk.pos_tag(token)
        all,idx = token_tag(token_pos)
        all = [[x] for x in all]
#         print(all)
        try:
            sm = []
            for d in all:
                model_dict = dict(model.wv.most_similar(d))
                sm.append([e for e in model_dict.keys() if model_dict[e] >= 0.75])
            idx_dic = []
            sm = defaultdict(dict)
            for f in (dict_s.keys()):
                if (f not in idx):
                    idx_dic.append(f)
                else:
                    for g in range(len(all[f])):
                        sm[g].update({all[f][g]:similarity(dict_s[f],all[f][g])})
                        
                        
            best = [[(h,i,sm[h][i]) for i in sm[h] if sm[h][i]>=0.63] for h in sm]
            print(l,best)


            
            default_s = dict_s
            
#             global all_str
            all_str = []
            
            
            for j in range(len(best)):
                for k in range(len(best[j])):
                    default_s[best[j][k][0]] = best[j][k][1]
                    all_str.append(' '.join(list(default_s.values())))
                    
#             distractors[l] = ','.join(all_str[:3])
            l += 1
#             print(l,all_str)
            
        except Exception as e:
            pass
#     print(distractors)
        

In [53]:
import_data(test_data)

2 [[(0, 'spoken', 1.0), (0, 'highly', 1.0)]]
3 [[(0, 'thermometer', 0.99999994)]]
4 [[(0, 'probably', 1.0), (0, 'impress', 1.0), (0, 'participants', 1.0), (0, 'deeply', 1.0)]]
5 [[(0, 'doctor', 1.0)]]
6 [[(0, 'rivers', 1.0)]]
7 [[(0, 'too', 1.0), (0, 'much', 1.0), (0, 'homework', 1.0)]]
8 [[(0, 'mass', 1.0), (0, 'extinction', 1.0)]]
9 [[(0, 'harmful', 1.0), (0, 'chemicals', 1.0)]]
10 [[(0, 'old', 1.0), (0, 'people', 1.0)]]
11 [[(0, 'stop', 1.0), (0, 'wasting', 1.0), (0, 'food', 1.0)]]
12 [[(0, 'sharks', 1.0), (0, 'are', 0.99999994), (0, 'dangerous', 1.0), (0, 'animals', 1.0)]]
13 [[(0, 'faces', 1.0), (0, 'danger', 0.99999994)]]
14 [[(0, 'more', 1.0), (0, 'people', 1.0)]]
15 [[(0, 'dinner', 1.0)]]
16 [[(0, 'throwing', 1.0), (0, 'stones', 1.0), (0, 'often', 1.0), (0, 'caused', 1.0), (0, 'injuries', 1.0)]]
17 [[(0, 'new', 1.0), (0, 'information', 1.0), (0, 'brings', 0.99999994)]]
18 [[(0, 'lives', 1.0), (0, 'alone', 1.0)]]
19 [[(0, 'picnics', 1.0)]]
20 [[(0, 'young', 1.0), (0, 'children',

137 [[(0, 'something', 1.0), (0, 'has', 1.0), (0, 'changed', 0.99999994)]]
138 [[(0, 'fast', 1.0), (0, 'rate', 1.0)]]
139 [[(0, 'conventional', 1.0), (0, 'playtime', 1.0)]]
140 [[(0, 'customs', 1.0)]]
141 []
142 []
143 [[(0, 'sleep', 1.0)]]
144 [[(0, 'avoiding', 1.0), (0, 'driving', 0.99999994), (0, 'cars', 1.0)]]
145 [[(0, 'keep', 1.0), (0, 'down', 1.0), (0, 'noise', 1.0)]]
146 [[(0, 'food', 1.0), (0, 'service', 1.0), (0, 'assistant', 1.0)]]
147 [[(0, 'liked', 1.0), (0, 'being', 1.0), (0, 'told', 1.0), (0, 'lies', 1.0)]]
148 [[(0, 'finding', 1.0), (0, 'toys', 1.0)]]
149 [[(0, 'keeping', 0.99999994), (0, 'close', 1.0), (0, 'enough', 1.0), (0, 'is', 1.0), (0, 'preferred', 1.0)]]
150 [[(0, 'junk', 1.0), (0, 'sleep', 1.0)]]
151 [[(0, 'stomach', 1.0), (0, 'diseases', 1.0)]]
152 [[(0, 'show', 1.0), (0, 'interest', 1.0)]]
153 [[(0, 'folic', 1.0), (0, 'acid', 1.0)]]
154 [[(0, 'has', 1.0), (0, 'limited', 1.0), (0, 'freedom', 1.0)]]
155 [[(0, 'nuclear', 1.0), (0, 'radiation', 1.0)]]
156 [[(0, '

275 [[(0, 'received', 1.0), (0, 'good', 1.0), (0, 'service', 1.0)]]
276 [[(0, 'mailed', 1.0), (0, 'programmes', 1.0)]]
277 [[(0, 'unreal', 0.99999994), (0, 'drugs', 1.0)]]
278 [[(0, 'read', 1.0), (0, 'international', 1.0), (0, 'business', 1.0), (0, 'newspapers', 1.0)]]
279 [[(0, 'choose', 1.0), (0, 'useful', 1.0), (0, 'classes', 1.0)]]
280 [[(0, 'die', 1.0), (0, 'earlier', 1.0)]]
281 [[(0, 'read', 1.0), (0, 'books', 1.0)]]
282 [[(0, 'glaciers', 1.0), (0, 'are', 0.99999994), (0, 'destructive', 1.0)]]
283 [[(0, 'prevent', 1.0), (0, 'illegal', 1.0), (0, 'business', 1.0)]]
284 [[(0, 'ancient', 0.99999994), (0, 'forests', 1.0)]]
285 [[(0, 'favorite', 1.0), (0, 'spring', 1.0), (0, 'activities', 1.0)]]
286 [[(0, 'sign', 1.0), (0, 'language', 1.0)]]
287 [[(0, 'exercise', 1.0), (0, 'often', 1.0)]]
288 [[(0, 'playing', 1.0), (0, 'football', 1.0)]]
289 [[(0, 'schoolmates', 1.0)]]
290 [[(0, 'hard', 1.0), (0, 'work', 0.99999994)]]
291 [[(0, 'suffered', 0.99999994), (0, 'heavy', 1.0), (0, 'financial